#Importing libraries

In [4]:
import numpy as np
import pandas as pd
import time
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from os.path import join
import os
from tqdm import tqdm

In [5]:
!nvidia-smi

Wed Jun  9 17:09:31 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
import keras
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, Lambda, MaxPool2D, BatchNormalization
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras import models, layers, optimizers
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.utils import class_weight
from keras.optimizers import SGD, RMSprop, Adam, Adagrad, Adadelta, RMSprop
from keras.models import Sequential, model_from_json
from keras.layers import Activation,Dense, Dropout, Flatten, Conv2D, MaxPool2D,MaxPooling2D,AveragePooling2D, BatchNormalization,GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras import backend as K
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.applications.inception_v3 import InceptionV3
import os
from glob import glob
import matplotlib.pyplot as plt
import random
import cv2
import pandas as pd
import numpy as np
import matplotlib.gridspec as gridspec
import seaborn as sns
import zlib
import itertools
import sklearn
import itertools
import scipy
import skimage
from skimage.transform import resize
import csv
from tqdm import tqdm
from sklearn import model_selection
from sklearn.model_selection import train_test_split, learning_curve,KFold,cross_val_score,StratifiedKFold
from sklearn.utils import class_weight
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

%matplotlib inline

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
from pathlib import Path

In [9]:
import tensorflow as tf

#Loading JPG Data

##TRAIN DATA

In [10]:

train_dir = Path( "/content/drive/MyDrive/Cs412Projectdata/train_data/")

In [11]:
train_dir.glob(r'**/*.jpg')


<generator object Path.glob at 0x7fd08aaa3450>

In [13]:
filepath = list(train_dir.glob(r'**/*.jpg'))

In [14]:
classes = list(map(lambda x: os.path.split(os.path.split(x)[0])[1],filepath))

In [15]:
df = pd.concat([pd.Series(filepath).astype(str), pd.Series(classes)], axis=1)
df.columns = ['Images', 'Image_label']

In [16]:
sample_size = 750
df = df.groupby("Image_label").sample(n=sample_size)

##TEST DATA

In [23]:

test_dir = Path("/content/drive/MyDrive/Cs412Projectdata/nonlabeled_test2/")


In [24]:
test_dir.glob(r'**/*.jpg')

<generator object Path.glob at 0x7fd02a3cff50>

In [25]:
filepath_Test = list(test_dir.glob(r'**/*.jpg'))

In [26]:
filepath_Test

[PosixPath('/content/drive/MyDrive/Cs412Projectdata/nonlabeled_test2/10342.jpg'),
 PosixPath('/content/drive/MyDrive/Cs412Projectdata/nonlabeled_test2/5009.jpg'),
 PosixPath('/content/drive/MyDrive/Cs412Projectdata/nonlabeled_test2/2285.jpg'),
 PosixPath('/content/drive/MyDrive/Cs412Projectdata/nonlabeled_test2/14139.jpg'),
 PosixPath('/content/drive/MyDrive/Cs412Projectdata/nonlabeled_test2/6232.jpg'),
 PosixPath('/content/drive/MyDrive/Cs412Projectdata/nonlabeled_test2/5685.jpg'),
 PosixPath('/content/drive/MyDrive/Cs412Projectdata/nonlabeled_test2/10414.jpg'),
 PosixPath('/content/drive/MyDrive/Cs412Projectdata/nonlabeled_test2/5367.jpg'),
 PosixPath('/content/drive/MyDrive/Cs412Projectdata/nonlabeled_test2/5976.jpg'),
 PosixPath('/content/drive/MyDrive/Cs412Projectdata/nonlabeled_test2/1953.jpg'),
 PosixPath('/content/drive/MyDrive/Cs412Projectdata/nonlabeled_test2/5728.jpg'),
 PosixPath('/content/drive/MyDrive/Cs412Projectdata/nonlabeled_test2/7339.jpg'),
 PosixPath('/content/driv

In [27]:
classes_test = list(map(lambda x: os.path.split(os.path.split(x)[0])[1],filepath_Test))

In [28]:
df_test = pd.concat([pd.Series(filepath_Test).astype(str), pd.Series(classes_test)], axis=1)
df_test.columns = ['Images', 'Image_label']

In [29]:
df_test.shape

(14500, 2)

#Splitting data and Image process

##TRAIN DATA

In [17]:
train_new, valid = train_test_split(df, test_size = 0.2, random_state = 42)
print(f"train set shape: {train_new.shape}")

print(f"validation set shape: {valid.shape}")

train set shape: (17400, 2)
validation set shape: (4350, 2)


In [18]:
train_datagen = ImageDataGenerator(rescale = 1.0/255.)


In [19]:
train_gen = train_datagen.flow_from_dataframe(dataframe = train_new,
                                              x_col = 'Images', y_col ='Image_label',
                                              target_size = (200,200), batch_size = 16, 
                                              class_mode = 'categorical', shuffle = True,featurewise_center=True )
val_gen = train_datagen.flow_from_dataframe(valid, 
                                            target_size=(200,200), x_col = 'Images', y_col ='Image_label', 
                                            class_mode='categorical',
                                            batch_size= 16, shuffle=False ,featurewise_center=True)


Found 17400 validated image filenames belonging to 29 classes.
Found 4350 validated image filenames belonging to 29 classes.


##TEST DATA

In [31]:
test_datagen = ImageDataGenerator(rescale = 1.0/255.)

In [32]:
test_gen = test_datagen.flow_from_dataframe(df_test,
                                           target_size = (200,200), x_col = 'Images', y_col ='Image_label',
                                            class_mode = 'categorical',
                                           batch_size = 16, shuffle = False,featurewise_center = True)

Found 14500 validated image filenames belonging to 1 classes.


In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

#Creating model

In [ ]:
base_model = tf.keras.applications.ResNet50V2(weights='imagenet', input_shape = (200,200,3),
                                                     include_top=False)

for layer in base_model.layers:
    layer.trainable = False
    
base_model.summary()    

In [22]:
model = Sequential()
model.add(base_model)

model.add(GlobalAveragePooling2D())
model.add(Dropout(0.2))
model.add(Dense(256,activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(29,activation="softmax"))


callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

history_onelayer = model.fit(train_gen, validation_data = val_gen, epochs = 5000, 
                    callbacks = [callbacks], verbose = 1)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/5000
1088/1088 [==============================] - 6172s 6s/step - loss: 1.2507 - accuracy: 0.6406 - val_loss: 0.1945 - val_accuracy: 0.9398
Epoch 2/5000
1088/1088 [==============================] - 66s 60ms/step - loss: 0.2484 - accuracy: 0.9208 - val_loss: 0.1086 - val_accuracy: 0.9671
Epoch 3/5000
1088/1088 [==============================] - 67s 61ms/step - loss: 0.1730 - accuracy: 0.9438 - val_loss: 0.0769 - val_accuracy: 0.9777
Epoch 4/5000
1088/1088 [==============================] - 68s 62ms/step - loss: 0.1455 - accuracy: 0.9518 - val_loss: 0.0885 - val_accuracy: 0.9694
Epoch 5/5000
1088/1088 [==============================] - 68s 62ms/step - loss: 0.1248 - accuracy: 0.9577 - val_loss: 0.0611 - val_accuracy: 0.9789
Epoch 6/5000
1088/1088 [==============================] - 68s 63ms/step - loss: 0.1182 - accuracy: 0.9623 - val_loss: 0.0544 - val_accuracy: 0.9818
Epoch 7/5000
1088/1088 [==============================] - 69s 63ms/step - loss: 0.0936 - accuracy: 0.9681 - val_

#Prediction

In [33]:
predictions = model.predict(test_gen)
predictions

array([[2.8201522e-10, 5.0363060e-17, 4.2517753e-13, ..., 1.6945346e-11,
        3.7170473e-13, 1.7063763e-05],
       [3.1388556e-09, 1.7838001e-15, 3.0615532e-15, ..., 1.4294701e-13,
        4.7790899e-15, 1.0071911e-07],
       [5.4667136e-19, 4.2568782e-20, 7.9186806e-14, ..., 1.9584471e-12,
        2.5528728e-17, 8.4698941e-06],
       ...,
       [9.9999845e-01, 1.4983343e-12, 6.9023897e-23, ..., 3.5922721e-21,
        5.7853497e-20, 2.3240623e-24],
       [1.0000000e+00, 1.6827906e-13, 2.4001952e-21, ..., 2.4795890e-22,
        8.0035210e-19, 1.4993266e-26],
       [1.0000000e+00, 5.7253827e-11, 4.7215867e-19, ..., 4.2103716e-20,
        1.3993783e-17, 8.1734556e-21]], dtype=float32)

In [34]:
preds = np.argmax(predictions, axis=1 )
labels = train_gen.class_indices
labels


{'A': 0,
 'B': 1,
 'C': 2,
 'D': 3,
 'E': 4,
 'F': 5,
 'G': 6,
 'H': 7,
 'I': 8,
 'J': 9,
 'K': 10,
 'L': 11,
 'M': 12,
 'N': 13,
 'O': 14,
 'P': 15,
 'Q': 16,
 'R': 17,
 'S': 18,
 'T': 19,
 'U': 20,
 'V': 21,
 'W': 22,
 'X': 23,
 'Y': 24,
 'Z': 25,
 'del': 26,
 'nothing': 27,
 'space': 28}

In [35]:
preds

array([24, 24, 24, ...,  0,  0,  0])

#Uploading

In [40]:
test_gen.filenames[1][56]

'/'

In [42]:

new = []
for each in test_gen.filenames:
  a = each[57::]
  a = int(a.strip(".jpg"))
  new.append(a)

In [43]:

labels = dict((v,k) for k,v in labels.items())
preds = [labels[k] for k in preds]

In [44]:

last = pd.DataFrame({'Id':new, 'Prediction':preds} )

last = last.sort_values("Id")

In [ ]:
last

In [45]:
last.to_csv("submission.csv" , index = False)